#### Heart Disease Model (WIP)

Descrição: O dataset de Heart Disease contém informações clínicas e demográficas de pacientes utilizadas para prever a presença de doença cardíaca. As classes representam o diagnóstico de doença cardíaca e variam de 0 a 4, onde 0 indica nenhuma doença e valores maiores indicam diferentes graus de presença da doença. O dataset possui um total de 303 instâncias.

### Imports

In [1]:

import numpy as np
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import wisardpkg as wp
from imblearn.under_sampling import RandomUnderSampler



### Carregando o Dataset

In [2]:
heart_disease = fetch_ucirepo(id=45)
X_original = heart_disease.data.features
y_original = heart_disease.data.targets.iloc[:, 0]


print(f"Dimensão inicial de X: {X_original.shape}")
#print(f"Colunas de X e seus tipos de dados:\n{X_original.dtypes}")
#print(f"\nPrimeiras 5 linhas de X:\n{X_original.head()}")
print(f"\nClasses de y:\n{y_original.value_counts()}")

Dimensão inicial de X: (303, 13)

Classes de y:
num
0    164
1     55
2     36
3     35
4     13
Name: count, dtype: int64


### Pré-processamento

#### Removendo nulos

In [3]:
print(X_original.isnull().sum()[X_original.isnull().sum() > 0])
X_original = X_original.replace('?', np.nan)
X_original['ca'] = X_original['ca'].fillna(X_original['ca'].mode()[0])
X_original['thal'] = X_original['thal'].fillna(X_original['thal'].mode()[0])

print("\nValores ausentes após preenchimento:")
print(X_original.isnull().sum()[X_original.isnull().sum() > 0])


ca      4
thal    2
dtype: int64

Valores ausentes após preenchimento:
Series([], dtype: int64)


### Binarização dos Atributos para o WiSARD usando o Termômetro

In [4]:
from module import termometro

X_encoded_parts = []
n_bits = 16 

numerical_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

# Binarizar colunas numéricas contínuas com Termômetro
for col in numerical_cols:
    X_encoded_parts.append(termometro.codificador_termometro(X_original[col].astype(float), n_bits))
    print(f"Coluna '{col}' binarizada com {n_bits} bits (Termômetro).")



X_final_binary = np.hstack(X_encoded_parts)
X_encoded_uint8 = X_final_binary.astype(np.uint8)

ModuleNotFoundError: No module named 'module'

#### One Hot Encoding para atributos categóricos

In [ ]:
categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

# Binarizar colunas categóricas com OneHotEncoder
for col in categorical_cols:
    # OneHotEncoder espera 2D array, então reshape a coluna
    onehot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    encoded_col = onehot_encoder.fit_transform(X_original[[col]].astype(str))
    X_encoded_parts.append(encoded_col.astype(np.uint8))
    print(f"Coluna '{col}' binarizada com OneHotEncoder (Dimensão: {encoded_col.shape[1]} bits).")

Coluna 'sex' binarizada com OneHotEncoder (Dimensão: 2 bits).
Coluna 'cp' binarizada com OneHotEncoder (Dimensão: 4 bits).
Coluna 'fbs' binarizada com OneHotEncoder (Dimensão: 2 bits).
Coluna 'restecg' binarizada com OneHotEncoder (Dimensão: 3 bits).
Coluna 'exang' binarizada com OneHotEncoder (Dimensão: 2 bits).
Coluna 'slope' binarizada com OneHotEncoder (Dimensão: 3 bits).
Coluna 'ca' binarizada com OneHotEncoder (Dimensão: 4 bits).
Coluna 'thal' binarizada com OneHotEncoder (Dimensão: 3 bits).


#### Transformando X e y para tipos aceitos pela WiSARD

In [ ]:
y_str = y_original.astype(str).tolist()
print(f"\nPrimeiros 5 rótulos de y (como strings): {y_str[:5]}")

X_final_binary = np.hstack(X_encoded_parts)
X_encoded_uint8 = X_final_binary.astype(np.uint8)

print(f"\nDimensão de X após todas as codificações: {X_encoded_uint8.shape}")
print(f"Tipo dos elementos de X_final_binary: {X_encoded_uint8.dtype}")
print("Primeiras 2 linhas de X_final_binary:\n", X_encoded_uint8[:2])



Primeiros 5 rótulos de y (como strings): ['0', '2', '1', '0', '0']

Dimensão de X após todas as codificações: (303, 103)
Tipo dos elementos de X_final_binary: uint8
Primeiras 2 linhas de X_final_binary:
 [[1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1
  0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 0
  0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 1 1 0 0 0 1 1 0 0 0 0 1 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1
  1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0
  0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 1 0 1 0 1 0 0 0 0 1 1 0 0]]


### Dividindo os Dados em Treino e Teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_final_binary, y_str, test_size=0.3, random_state=42, stratify=y_str
)

print(f"Tamanho do conjunto de treino (X_train): {X_train.shape[0]} amostras, {X_train.shape[1]} bits")
print(f"Tamanho do conjunto de teste (X_test): {X_test.shape[0]} amostras, {X_test.shape[1]} bits")
print(f"Tipo dos elementos de X_train: {X_train.dtype}")

Tamanho do conjunto de treino (X_train): 212 amostras, 103 bits
Tamanho do conjunto de teste (X_test): 91 amostras, 103 bits
Tipo dos elementos de X_train: uint8


### Treinando o Modelo WiSARD

In [ ]:
addressSize = 8
wsd = wp.Wisard(addressSize, ignoreZero=False, verbose=True)

print(f"WiSARD inicializado com addressSize={addressSize}")
wsd.train(X_train, y_train)

predictions_str = wsd.classify(X_test)

y_pred_np = np.array(predictions_str)
y_test_np = np.array(y_test) 

WiSARD inicializado com addressSize=8
training 212 of 212
classifying 91 of 91


### Relatório de Classificação

In [ ]:
print(classification_report(
    y_test_np, y_pred_np,
    labels=np.unique(y_test)
))

              precision    recall  f1-score   support

           0       0.72      0.98      0.83        49
           1       0.55      0.35      0.43        17
           2       0.29      0.18      0.22        11
           3       0.17      0.10      0.12        10
           4       0.00      0.00      0.00         4

    accuracy                           0.63        91
   macro avg       0.34      0.32      0.32        91
weighted avg       0.54      0.63      0.57        91



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


### Conclusões

O modelo WiSARD não performou bem com o dataset de heart disease em nenhuma das métricas de avaliação.